# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [191]:
""" # Please remove the triple quotes in this line and last line +; if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycrypto
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

In [192]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode

key=b'1234567891234567'
nonce = get_random_bytes(15)    # instead of getting a random nonce
message = "Information security is a great course !".encode() # convert the string to bytes using encode function

# Encryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
ciphertext, mac = cipher.encrypt_and_digest(message)
# Decryption
cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
plaintext = cipher.decrypt_and_verify(ciphertext, mac)

print("PlainText before Encryption: "+message.decode())
print("Ciphertext: "+b64encode(ciphertext).decode())
print("Back to PlainText: "+plaintext.decode())

PlainText before Encryption: Information security is a great course !
Ciphertext: 53b5PeH2ZthfJbiXu51JxMk0UT9PwuMhrPtNP9xKCB/pVLCw9t/ZGg==
Back to PlainText: Information security is a great course !


## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).
We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).
Now we can encrypt the same message twice and see what we get in the ciphertext:

# AES USING ECB (ELECTRONIC CODE BOOK)

In [193]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567'
    message = "Information security is a great course, I love it !".encode() # convert the string to bytes using encode function

    # Encryption
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_ECB)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext "+str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext 0 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 0 : Information security is a great course, I love it !

Ciphertext 1 : TLHenMawihtNigFCM8Ub0mDIBGYS4iN33JaUOLJ6/xeW9hrNVPnMM6ooHIZdZH9P3JUNHQEy2+UwjdMz8WQNmA==
Back to PlainText 1 : Information security is a great course, I love it !



This is not a desirable outcome. If I want to send the same message twice, I really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. A better mode is the Cipher block chaining (CBC):

# AES USING CBC (CIPHER BLOCK CHAINING)

In [194]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode
for x in range(2):
    #Key size can be 16, 24 or 32 bytes 
    # key = get_random_bytes(16)      # A 16 byte key for AES-128
    key=b'1234567891234567' # 16 bytes
    m = "Information Security is the best course, I love it !" 
    message=m.encode()      # convert the string to bytes using encode function
    # Encryption
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext= cipher.encrypt(pad(message, AES.block_size)) # Block size= 16 bytes
    iv = cipher.iv
    # Decryption
    decrypt_cipher = AES.new(key, AES.MODE_CBC,iv)
    plaintext =  decrypt_cipher.decrypt(ciphertext)
    print("Ciphertext "+str(x)+" : "+b64encode(ciphertext).decode())
    print("Back to PlainText "+str(x)+" : "+plaintext.decode()+'\n')

Ciphertext 0 : +yGw/SSz381Yg7ATV02Yoa11k06fZoHBD8cOhNuzGwqhMGBoW4Q48Hvglh0CL40r61ZcofHYcy95ohKIRugMNg==
Back to PlainText 0 : Information Security is the best course, I love it !

Ciphertext 1 : UGsYmQbKmyeyqFcRVeIOxfYZ8q+Nf9Lv/tOmlsc3P2haUJH5iKzCtS9s5VtP32QUZKUVK2TFEbCAx2pg5Wuiww==
Back to PlainText 1 : Information Security is the best course, I love it !



In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

## Size of ciphertext <a class="anchor" id="size"></a>

In [199]:
import sys
print("CipherText-Data"+'\n'+'-'*100)
print("CipherText: "+str(b64encode(ciphertext).decode()))
print("Length of CipherText: "+str(len(ciphertext)))
print("Size of CipherText: "+str(sys.getsizeof(ciphertext))+'\n')
print("PlainText-Data"+'\n'+'-'*100)
print("PlainText: "+m)
print("Length of PlainText: "+str(len(m)))
print("Size of PlainText: "+str(sys.getsizeof(m))+'\n')

CipherText-Data
----------------------------------------------------------------------------------------------------
CipherText: UGsYmQbKmyeyqFcRVeIOxfYZ8q+Nf9Lv/tOmlsc3P2haUJH5iKzCtS9s5VtP32QUZKUVK2TFEbCAx2pg5Wuiww==
Length of CipherText: 64
Size of CipherText: 97

PlainText-Data
----------------------------------------------------------------------------------------------------
PlainText: Information Security is the best course, I love it !
Length of PlainText: 52
Size of PlainText: 101



The ciphertext has a size of 97 bytes, while the plaintext has a size of 101 bytes

# Conclusion

The biggest difference observed between the 2 modes (ECB and CBC) is that in the example of ECB we had a set key (just for the sake of this project) that produces the same ciphertext everytime. This is not good, because if someone knows or learns a certain pattern in our plaintext (message), it can be easily decrypted. In the CBC example however, while also setting a fixed key (again just for this example) it produced 2 different ciphertexts. This is better because it adds another layer of security

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 

In [196]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"This bonus is not easy !")
print("Key= "+str(key.decode())+'\n')
print("Token= "+str(token.decode())+'\n')
print("After decryption= "+str(f.decrypt(token).decode()))

Key= caaKC-PGNFB9xVSOuf4rvddD9IpPzxoXkIhYuA-mZuY=

Token= gAAAAABjeMY-z8nmX7i-Wta6Jlwg6JlTxcoLUEynPQBG4TIJGw6OZba3WQzLPT33_xIlJGmsly_IhhBlwv--FW_9pDvTY7SA4VCQ3INeT-SEC7Vh5i4OYZw=

After decryption= This bonus is not easy !


Just to double check that our code is running correctly, we will have a fixed key and token and run them on the [Fernet](https://asecuritysite.com/encryption/ferdecode) website and compare the results

In [197]:
from cryptography.fernet import Fernet
# key = Fernet.generate_key()
key=b'e-n2_qyYvwWwXksiSBXUYVvs-eevERo78RuPxwbr-6Y='
f = Fernet(key)
# token = f.encrypt(b"This bonus is not easy !")
token=b'gAAAAABjd9Rn6g-s8YNq0gzFNcxUyMdVrETC5-Apzi_EcqJ0pLAQI_jRx6JJDQap7YO8jRC74wnH1AFELYbQyQsVVCpbQC1tD6BhFsFBA4UBEaJKdEFXG-A='
print("Key= "+str(key.decode())+'\n')
print("Token= "+str(token.decode())+'\n')
print("After decryption = "+str(f.decrypt(token).decode()))

Key= e-n2_qyYvwWwXksiSBXUYVvs-eevERo78RuPxwbr-6Y=

Token= gAAAAABjd9Rn6g-s8YNq0gzFNcxUyMdVrETC5-Apzi_EcqJ0pLAQI_jRx6JJDQap7YO8jRC74wnH1AFELYbQyQsVVCpbQC1tD6BhFsFBA4UBEaJKdEFXG-A=

After decryption = This bonus is not easy !


The results are:

<img src="Fernet.png" style="width:900px"/>

As we can see the key and token resulted in the right plaintext to be decoded, which means that our code is running correctly 